In [1]:
from osgeo import gdal
import os
import numpy as np
import cv2
import re
import sentinel1decoder

In [2]:
def start_points(size, split_size, overlap=0): # overlap = % of overlap between tiles, both rows and columns 
    points = [0]
    stride = int(split_size * (1-overlap))
    counter = 1
    while True:
        pt = stride * counter
        if pt + split_size >= size:
            points.append(size - split_size)
            break
        else:
            points.append(pt)
        counter += 1
    return points


def create_patches_labels(L1, folder, i):     
    patch_dir = os.path.join(folder, 'L1_tile')
    if not os.path.isdir(patch_dir):
        os.mkdir(patch_dir)
        
    img_h, img_w = L1.shape
    split_width = 3034*2 # define tile size 
    split_height = 916*2

    X_points = start_points(img_w, split_width, 0)
    Y_points = start_points(img_h, split_height, 0)
    name = 'splitted'
    
    tot = []
    for s in Y_points:
        for j in X_points:
            split = L1[s:s+split_height, j:j+split_width]
            patch_name = f'patch_{name}_{i}_{s:02}_{j:02}.npy'
            split = split.astype(np.float32)
            np.save(os.path.join(patch_dir, patch_name), split)
            tot.append(split)        
    return tot



def create_patches(re, im, folder, i):      
    patch_dir = os.path.join(folder, 'L0_tile')
    if not os.path.isdir(patch_dir):
        os.mkdir(patch_dir)
        
    img_h, img_w = re.shape #or im
    split_width = 3034*2
    split_height = 916*2

    X_points = start_points(img_w, split_width, 0)
    Y_points = start_points(img_h, split_height, 0)
    name = 'splitted'
    
    tot = []
    for s in Y_points:
        for j in X_points:
            split_re = re[s:s+split_height, j:j+split_width]

            split_im = im[s:s+split_height, j:j+split_width]
            
            split = cv2.merge([split_re, split_im])
            split = split.astype(np.float32)
            
            patch_name = f'patch_{name}_{i}_{s:02}_{j:02}.npy'
            np.save(os.path.join(patch_dir, patch_name), split)
            tot.append(split)
        
    return tot

In [26]:
path = "L0"
L0_files =[]
for root, dirs, files in os.walk("L0/"):
    for file in files:
        strin1 = 'annot'
        string2 = 'index'
        if file.endswith(".dat") and strin1 not in file and string2 not in file:
            L0_files.append(root + '/' + file)
print(len(L0_files))



path = "L1"
L1_files = []
for root, dirs, files in os.walk(path):
    for file in files:
        check = str(os.path.join(root, file))
        if(file.endswith("001.tiff")):
            L1_files.append(os.path.join(root,file))
print(L1_files)

2
['L1\\s1a-s1-slc-vh-20230704t020728-20230704t020752-049267-05ec9b-001.tiff']


In [19]:
l0file = sentinel1decoder.Level0File(l[0])

In [21]:
l0file.packet_metadata

Packet Version Number  Packet Type  \
Azimuth Block Number Packet Number                                       
1                    0                                  0            0   
                     1                                  0            0   
                     2                                  0            0   
                     3                                  0            0   
                     4                                  0            0   
...                                                   ...          ...   
16                   47586                              0            0   
                     47587                              0            0   
                     47588                              0            0   
                     47589                              0            0   
                     47590                              0            0   

                                    Secondary Header Flag  PID  PCAT  \
Azimuth Block Number Packet Number                                     
1                    0                                  1   65    12   
                     1                                  1   65    12   
                     2                                  1   65    12   
                     3                                  1   65    12   
                     4                                  1   65    12   
...                                                   ...  ...   ...   
16                   47586                              1   65    12   
                     47587                              1   65    12   
                     47588                              1   65    12   
                     47589                              1   65    12   
                     47590                              1   65    12   

                                    Sequence Flags  Packet Sequence Count  \
Azimuth Block Number Packet Number                                          
1                    0                           3                      0   
                     1                           3                      1   
                     2                           3                      2   
                     3                           3                      3   
                     4                           3                      4   
...                                            ...                    ...   
16                   47586                       3                     34   
                     47587                       3                     35   
                     47588                       3                     36   
                     47589                       3                     37   
                     47590                       3                     38   

                                    Packet Data Length  Coarse Time  \
Azimuth Block Number Packet Number                                    
1                    0                           32338   1372471665   
                     1                           32338   1372471665   
                     2                           32338   1372471665   
                     3                           32338   1372471665   
                     4                           32338   1372471665   
...                                                ...          ...   
16                   47586                       32326   1372471691   
                     47587                       32326   1372471691   
                     47588                       32326   1372471691   
                     47589                       32326   1372471691   
                     47590                       32326   1372471691   

                                    Fine Time  ...       SWL  SAS SSB Flag  \
Azimuth Block Number Packet Number             ...                           
1                    0               0.618004  ...  0.

In [22]:
l0file.ephemeris

,X-axis position ECEF,Y-axis position ECEF,Z-axis position ECEF,X-axis velocity ECEF,Y-axis velocity ECEF,Z-axis velocity ECEF,POD Solution Data Timestamp,Q0 Attitude Quaternion,Q1 Attitude Quaternion,Q2 Attitude Quaternion,Q3 Attitude Quaternion,Omega-X Angular Rate,Omega-Y Angular Rate,Omega-Z Angular Rate,Attitude Data Timestamp
0,3.753654e+06,5.976353e+06,-535957.120673,1047.500610,-1312.178467,-7408.310059,1.372472e+09,0.319740,0.617579,0.182630,-0.694988,-0.000056,-0.000911,-0.000538,1.372472e+09
1,3.753654e+06,5.976353e+06,-535957.120673,1047.500610,-1312.178467,-7408.310059,1.372472e+09,0.319740,0.617579,0.182630,-0.694988,-0.000056,-0.000911,-0.000538,1.372472e+09
2,3.753654e+06,5.976353e+06,-535957.120673,1047.500610,-1312.178467,-7408.310059,1.372472e+09,0.319740,0.617579,0.182630,-0.694988,-0.000056,-0.000911,-0.000538,1.372472e+09
3,3.753654e+06,5.976353e+06,-535957.120673,1047.500610,-1312.178467,-7408.310059,1.372472e+09,0.319740,0.617579,0.182630,-0.694988,-0.000056,-0.000911,-0.000538,1.372472e+09
4,3.753654e+06,5.976353e+06,-535957.120673,1047.500610,-1312.178467,-7408.310059,1.372472e+09,0.319740,0.617579,0.182630,-0.694988,-0.000056,-0.000911,-0.000538,1.372472e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
733,3.778462e+06,5.941414e+06,-720954.519660,936.924316,-1482.698364,-7390.579590,1.372472e+09,0.317595,0.608130,0.183631,-0.703982,-0.000064,-0.000909,-0.000537,1.372472e+09
734,3.778462e+06,5.941414e+06,-720954.519660,936.924316,-1482.698364,-7390.579590,1.372472e+09,0.317595,0.608130,0.183631,-0.703982,-0.000064,-0.000909,-0.000537,1.372472e+09
735,3.778462e+06,5.941414e+06,-720954.519660,936.924316,-1482.698364,-7390.579590,1.372472e+09,0.317595,0.608130,0.183631,-0.703982,-0.000064,-0.000909,-0.000537,1.372472e+09
736,3.778462e+06,5.941414e+06,-720954.519660,936.924316,-1482.698364,-7390.579590,1.372472e+09,0.317595,0.608130,0.183631,-0.703982,-0.000064,-0.000909,-0.000537,1.372472e+09
